## Packages

In [22]:
import pandas as pd
import numpy as np
import pickle as pkl

import plotly.graph_objects as go
from plotly.subplots import make_subplots
template='plotly_white'

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Helper functions

In [23]:
def get_metrics(y_test, y_pred, return_=False):
    # rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    residuals = y_test - y_pred
    res_mean = np.mean(residuals)
    res_std = np.std(residuals)

    print(f'R-squared: {round(r2,3)}')
    print(f'Mean Absolute Error: {"{:,.2f}".format(mae)}')
    # print(f'Root Mean Squared Error: {round(rmse,3)}')
    print(f'Residuals Mean: {"{:,.2f}".format(res_mean)}, Residuals Std: {"{:,.2f}".format(res_std)}')

    if return_:
        return r2, mae, res_mean, res_std

def make_res_plots(y_test, y_pred):
  residuals = y_test-y_pred

  fig = make_subplots(rows=1, cols=2, subplot_titles=('Residuals Plot', 'Residuals Histogram'))
  fig.add_trace(go.Scatter(x=residuals.index, y=residuals, mode='markers', marker={'color':'#1f77b4', 'opacity':0.6}, hoverinfo='y'), row=1, col=1)
  fig.add_hline(y=0, row=1, col=1, line={'color':'red', 'dash':'dot'})
  fig.add_trace(go.Histogram(x=residuals, marker={'color':'#1f77b4'}), row=1, col=2)
  fig.add_vrect(x0=np.mean(residuals)-np.std(residuals), x1=np.mean(residuals)+np.std(residuals), row=1, col=2,
                annotation_text='68%', annotation_position='top right',
                fillcolor='green', opacity=0.25, line_width=0)

  fig.update_layout(showlegend=False, template=template)
  fig.update_xaxes(showgrid=False, showline=False, zeroline=False)
  fig.update_yaxes(showgrid=False, showline=False, zeroline=False)
  fig.show()

def save_model(model, filename, path):
  SAVE_PATH = path + filename + '.pkl'
  with open(SAVE_PATH, 'wb') as f:
    pkl.dump(model, f)
    f.close()
  return 'model saved.'

# Load & Process data

In [17]:
MODEL_SAVE_PATH = r'C:/Users/Nick/Documents/Data Science/Personal projects/car_gr/models/'

DATA_PATH = r'C:/Users/Nick/Documents/Data Science/Personal projects/car_gr/data/clean/'
FILE_NAME = 'data_clean_20240509.csv'

data = pd.read_csv(DATA_PATH+FILE_NAME, sep=';')
# print(f'{data.shape[0]} rows, {data.shape[1]} attributes')

In [18]:
data_processed = pd.get_dummies(data, prefix_sep = '_')

X = data_processed.drop('Price', axis=1)
y = data_processed['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=123)

# Models building  
[*State of Competitive ML report 2023*](https://mlcontests.com/state-of-competitive-machine-learning-2023/)

## RandomForestRegressor

In [16]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
y_pred_rfr = rfr.predict(X_test)
get_metrics(y_test, y_pred_rfr)

R-squared: 0.868
Mean Absolute Error: 2,379.65
Residuals Mean: -77.86, Residuals Std: 6,230.62


### Hyperparameter Tuning  
Using randomized search to define a starting point for the exhaustive grid search with 3-fold cross validation

In [41]:
# Apply the results of randomized search to see if the model can perform better using the exhaustive search
param_grid = {'n_estimators' : [1_000], 'max_depth' : [18,19,20,21,22,23,24,25], 'max_features':[20]}

gsearch = GridSearchCV(estimator=RandomForestRegressor(n_jobs=-1), param_grid=param_grid, n_jobs=-1, cv=5, scoring='neg_mean_absolute_error')
gsearch.fit(X_train, y_train)
score = -gsearch.best_score_
print(f'Best score on training data (MAE): {"{:,.2f}".format(score)}')
print(f'Best parameters: {gsearch.best_params_}')

best_model = gsearch.best_estimator_
best_model.fit(X_train, y_train)
y_pred_best_model = best_model.predict(X_test)
get_metrics(y_test, y_pred_best_model)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best score (neg_mae) : 2358.3033167951326
R-squared: 0.884
Mean Absolute Error: 2,278.15
Residuals Mean: -41.10, Residuals Std: 5,845.56


In [25]:
rfr_best = RandomForestRegressor(max_depth=25, max_features=20, n_estimators=1000)
rfr_best.fit(X_train, y_train)
y_pred_rfr = rfr_best.predict(X_test)
get_metrics(y_test, y_pred_rfr)

R-squared: 0.884
Mean Absolute Error: 2,285.04
Residuals Mean: -45.60, Residuals Std: 5,858.80


In [26]:
save_model(rfr_best, 'RandomForest_tuned', MODEL_SAVE_PATH)

'model saved.'

## LightGBM

In [20]:
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(boosting_type='gbdt' , n_jobs=-1, force_col_wise=True)
lgbm.fit(X_train, y_train)
y_pred_lgbm = lgbm.predict(X_test)
get_metrics(y_test, y_pred_lgbm)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 845
[LightGBM] [Info] Number of data points in the train set: 14643, number of used features: 53
[LightGBM] [Info] Start training from score 15655.535136
R-squared: 0.857
Mean Absolute Error: 2,538.64
Residuals Mean: 4.41, Residuals Std: 6,482.59


### Hyperparameter Tuning

In [65]:
import optuna

def objective(trial):
  learning_rate = trial.suggest_float('learning_rate', 0.001, 0.3)
  max_depth = trial.suggest_int('max_depth', 2, 30)
  n_estimators = trial.suggest_int('n_estimator', 100, 10_000)


  model = LGBMRegressor(boosting_type='gbdt', learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators, n_jobs=-1, force_col_wise=True, verbose=-1)
  model.fit(X_train, y_train)
  y_pred_model = model.predict(X_test)

  return mean_absolute_error(y_test, y_pred_model)

study = optuna.create_study(study_name='LGBM_Optimization', direction='minimize')
study.optimize(objective, n_trials=3)

print(f'Best parameters from study: {study.best_params}')

[I 2024-05-19 15:03:23,640] A new study created in memory with name: LGBM_Optimization
[I 2024-05-19 15:03:32,391] Trial 0 finished with value: 2495.640300384332 and parameters: {'learning_rate': 0.2237292798623292, 'max_depth': 15, 'n_estimator': 2957}. Best is trial 0 with value: 2495.640300384332.
[I 2024-05-19 15:03:46,452] Trial 1 finished with value: 2598.805954084309 and parameters: {'learning_rate': 0.29717278284273263, 'max_depth': 9, 'n_estimator': 8412}. Best is trial 0 with value: 2495.640300384332.
[I 2024-05-19 15:03:52,269] Trial 2 finished with value: 2500.19426888577 and parameters: {'learning_rate': 0.18222490892083656, 'max_depth': 10, 'n_estimator': 3998}. Best is trial 0 with value: 2495.640300384332.


Best parameters from study: {'learning_rate': 0.2237292798623292, 'max_depth': 15, 'n_estimator': 2957}


In [66]:
lgmb_tuned = LGBMRegressor(boosting_type='gbdt', n_estimator=2957, learning_rate=0.2237292798623292, max_depth=15, n_jobs=-1, force_col_wise=True, verbose=-1)
lgmb_tuned.fit(X_train, y_train)
y_pred_lgmb_tuned = lgmb_tuned.predict(X_test)
get_metrics(y_test, y_pred_lgmb_tuned)

R-squared: 0.869
Mean Absolute Error: 2,426.14
Residuals Mean: -17.27, Residuals Std: 6,218.60


## XGBRegressor

In [58]:
from xgboost import XGBRegressor
xgb = XGBRegressor(n_jobs=-1)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
get_metrics(y_test, y_pred_xgb)

R-squared: 0.862
Mean Absolute Error: 2,443.88
Residuals Mean: -22.49, Residuals Std: 6,374.08


## CatBoost

In [72]:
from catboost import CatBoostRegressor
cb_reg = CatBoostRegressor(verbose=False)
cb_reg.fit(X_train, y_train)
y_pred_cb_reg = xgb.predict(X_test)
get_metrics(y_test, y_pred_cb_reg)

R-squared: 0.862
Mean Absolute Error: 2,443.88
Residuals Mean: -22.49, Residuals Std: 6,374.08


# Documentation

## Models Metrics [2024/05/19]
  
**Random Forest**  
Untuned:
>R-squared: 0.868  
Mean Absolute Error: 2,366.84  
Residuals Mean: -55.98, Residuals Std: 6,231.00  

Tuned:  
>Best parameters: {'max_depth': 25, 'max_features': 20, 'n_estimators': 1000}  
R-squared: 0.884  
Mean Absolute Error: 2,278.15  
Residuals Mean: -41.10, Residuals Std: 5,845.56

**LightGBM**  
Untuned:  
>R-squared: 0.857  
Mean Absolute Error: 2,538.64  
Residuals Mean: 4.41, Residuals Std: 6,482.59  

Tuned:  


**XGBoost**  
Untuned:
>R-squared: 0.862  
Mean Absolute Error: 2,443.88  
Residuals Mean: -22.49, Residuals Std: 6,374.08  

Tuned:  

**CatBoost**
Untuned:  
>R-squared: 0.862  
Mean Absolute Error: 2,443.88  
Residuals Mean: -22.49, Residuals Std: 6,374.08  


# SHAP values

In [69]:
import shap